ARIMA Modeling

In [0]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import warnings
warnings.filterwarnings("ignore")

!pip install pyramid-arima
from pyramid.arima import auto_arima

     |████████████████████████████████| 604kB 5.1MB/s 


In [0]:
calendar = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/calendar.csv')
train_val_date=calendar.loc[:1912,'date']

In [0]:
train_val_date

0       2011-01-29
1       2011-01-30
2       2011-01-31
3       2011-02-01
4       2011-02-02
           ...    
1908    2016-04-20
1909    2016-04-21
1910    2016-04-22
1911    2016-04-23
1912    2016-04-24
Name: date, Length: 1913, dtype: object

In [0]:
CA_sale = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/CA_sale.pkl')
# TX_sale = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/TX_sale.pkl')
# WI_sale = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/Kaggle_data/m5-forecasting-accuracy/WI_sale.pkl')

In [0]:
CA_sale

,id,d,value,wm_yr_wk,sell_price,National,Sporting,Cultural,Religious,LentStart,StPatricksDay,Halloween,Eid al-Fitr,LaborDay,Ramadan starts,NBAFinalsStart,SuperBowl,ValentinesDay,LentWeek2,NBAFinalsEnd,Thanksgiving,Cinco De Mayo,EidAlAdha,Christmas,VeteransDay,OrthodoxChristmas,Purim End,Mother's day,MartinLutherKingDay,PresidentsDay,Easter,NewYear,Father's day,IndependenceDay,Chanukah End,Pesach End,ColumbusDay,MemorialDay,OrthodoxEaster,snap_CA,snap_TX,snap_WI,year,month,wday
0,HOBBIES_1_001_CA_1,1,0,11101,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011,1,1
1,HOBBIES_1_002_CA_1,1,0,11101,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011,1,1
2,HOBBIES_1_003_CA_1,1,0,11101,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011,1,1
3,HOBBIES_1_004_CA_1,1,0,11101,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011,1,1
4,HOBBIES_1_005_CA_1,1,0,11101,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23330943,FOODS_3_823_CA_4,1913,0,11613,2.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2
23330944,FOODS_3_824_CA_4,1913,1,11613,2.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2
23330945,FOODS_3_825_CA_4,1913,0,11613,3.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2
23330946,FOODS_3_826_CA_4,1913,4,11613,1.28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,4,2


In [0]:
cap=CA_sale.pivot_table(index='d',columns='id',values='value')

In [0]:
print(cap.shape)
cap.index = train_val_date
cap.index = pd.to_datetime(cap.index)
cap.head() 

(1913, 12196)


id,FOODS_1_001_CA_1,FOODS_1_001_CA_2,FOODS_1_001_CA_3,FOODS_1_001_CA_4,FOODS_1_002_CA_1,FOODS_1_002_CA_2,FOODS_1_002_CA_3,FOODS_1_002_CA_4,FOODS_1_003_CA_1,FOODS_1_003_CA_2,FOODS_1_003_CA_3,FOODS_1_003_CA_4,FOODS_1_004_CA_1,FOODS_1_004_CA_2,FOODS_1_004_CA_3,FOODS_1_004_CA_4,FOODS_1_005_CA_1,FOODS_1_005_CA_2,FOODS_1_005_CA_3,FOODS_1_005_CA_4,FOODS_1_006_CA_1,FOODS_1_006_CA_2,FOODS_1_006_CA_3,FOODS_1_006_CA_4,FOODS_1_008_CA_1,FOODS_1_008_CA_2,FOODS_1_008_CA_3,FOODS_1_008_CA_4,FOODS_1_009_CA_1,FOODS_1_009_CA_2,FOODS_1_009_CA_3,FOODS_1_009_CA_4,FOODS_1_010_CA_1,FOODS_1_010_CA_2,FOODS_1_010_CA_3,FOODS_1_010_CA_4,FOODS_1_011_CA_1,FOODS_1_011_CA_2,FOODS_1_011_CA_3,FOODS_1_011_CA_4,...,HOUSEHOLD_2_507_CA_1,HOUSEHOLD_2_507_CA_2,HOUSEHOLD_2_507_CA_3,HOUSEHOLD_2_507_CA_4,HOUSEHOLD_2_508_CA_1,HOUSEHOLD_2_508_CA_2,HOUSEHOLD_2_508_CA_3,HOUSEHOLD_2_508_CA_4,HOUSEHOLD_2_509_CA_1,HOUSEHOLD_2_509_CA_2,HOUSEHOLD_2_509_CA_3,HOUSEHOLD_2_509_CA_4,HOUSEHOLD_2_510_CA_1,HOUSEHOLD_2_510_CA_2,HOUSEHOLD_2_510_CA_3,HOUSEHOLD_2_510_CA_4,HOUSEHOLD_2_511_CA_1,HOUSEHOLD_2_511_CA_2,HOUSEHOLD_2_511_CA_3,HOUSEHOLD_2_511_CA_4,HOUSEHOLD_2_512_CA_1,HOUSEHOLD_2_512_CA_2,HOUSEHOLD_2_512_CA_3,HOUSEHOLD_2_512_CA_4,HOUSEHOLD_2_513_CA_1,HOUSEHOLD_2_513_CA_2,HOUSEHOLD_2_513_CA_3,HOUSEHOLD_2_513_CA_4,HOUSEHOLD_2_514_CA_1,HOUSEHOLD_2_514_CA_2,HOUSEHOLD_2_514_CA_3,HOUSEHOLD_2_514_CA_4,HOUSEHOLD_2_515_CA_1,HOUSEHOLD_2_515_CA_2,HOUSEHOLD_2_515_CA_3,HOUSEHOLD_2_515_CA_4,HOUSEHOLD_2_516_CA_1,HOUSEHOLD_2_516_CA_2,HOUSEHOLD_2_516_CA_3,HOUSEHOLD_2_516_CA_4
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-01-29,3,2,1,0,0,2,1,0,0,8,1,0,0,0,0,0,3,8,10,2,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,4,5,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0
2011-01-30,0,0,2,1,1,1,0,1,0,2,1,1,0,0,0,0,9,1,3,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,4,9,0,0,1,0,0,0,0,0,0,1,1,2,2,0,0,0,0,1,0,1,0
2011-01-31,0,0,1,1,0,1,1,2,0,2,1,0,0,0,0,0,3,1,2,0,3,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,1,1,4,0,0,0,0,0,1,2,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2011-02-01,1,0,1,1,1,0,2,1,0,1,3,0,0,0,0,0,3,0,10,2,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,1,0,4,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2011-02-02,4,1,1,1,1,2,0,0,1,0,0,1,0,0,0,0,0,5,1,1,7,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,6,1,0,0,0,0,1,0,3,2,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0


In [0]:
cap.columns

Index(['FOODS_1_001_CA_1', 'FOODS_1_001_CA_2', 'FOODS_1_001_CA_3',
       'FOODS_1_001_CA_4', 'FOODS_1_002_CA_1', 'FOODS_1_002_CA_2',
       'FOODS_1_002_CA_3', 'FOODS_1_002_CA_4', 'FOODS_1_003_CA_1',
       'FOODS_1_003_CA_2',
       ...
       'HOUSEHOLD_2_514_CA_3', 'HOUSEHOLD_2_514_CA_4', 'HOUSEHOLD_2_515_CA_1',
       'HOUSEHOLD_2_515_CA_2', 'HOUSEHOLD_2_515_CA_3', 'HOUSEHOLD_2_515_CA_4',
       'HOUSEHOLD_2_516_CA_1', 'HOUSEHOLD_2_516_CA_2', 'HOUSEHOLD_2_516_CA_3',
       'HOUSEHOLD_2_516_CA_4'],
      dtype='object', name='id', length=12196)

In [0]:
# from statsmodels.tsa.seasonal import seasonal_decompose
# result = seasonal_decompose(pd.DataFrame(cap['FOODS_1_001_CA_1']), model= 'multiplicative')
# fig=result.plot()
# fig.set_size_inches(40,30)
# fig.show()

ValueError: ignored

In [0]:
t=pd.DataFrame(cap['FOODS_1_001_CA_1'])
t[t['FOODS_1_001_CA_1'] ==0]

,FOODS_1_001_CA_1
date,
2011-01-30,0
2011-01-31,0
2011-02-04,0
2011-02-06,0
2011-02-07,0
...,...
2016-04-12,0
2016-04-15,0
2016-04-17,0


In [0]:
stepwise_model = auto_arima(t, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=6280.794, BIC=6308.542, Fit time=37.843 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=8799.309, BIC=8810.408, Fit time=0.199 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=7661.258, BIC=7683.457, Fit time=3.072 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=6282.931, BIC=6305.129, Fit time=16.964 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=6274.877, BIC=6308.174, Fit time=35.994 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 0, 12); AIC=6941.637, BIC=6969.385, Fit time=21.388 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=6275.036, BIC=6313.883, Fit time=65.883 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=7519.014, BIC=7541.212, Fit time=8.713 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(2, 1, 2, 12); AIC=6274.816, BIC=6319.213, Fit time=95.535 seconds
Fit ARIMA: order=(0, 1

In [0]:
stepwise_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                 1913
Model:             SARIMAX(0, 1, 1)x(2, 1, 2, 12)   Log Likelihood               -3129.390
Date:                            Tue, 02 Jun 2020   AIC                           6272.780
Time:                                    12:05:13   BIC                           6311.627
Sample:                                         0   HQIC                          6287.081
                                           - 1913                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   2.936e-05   7.02e-05      0.418      0.676      -0.000       0.000
ma.L1         -0.9572      0.006   -154.245      0.000      -0.969      -0.945
ar.S.L12      -0.8821      0.406     -2.171      0.030      -1.679      -0.086
ar.S.L24      -0.0340      0.024     -1.436      0.151      -0.080       0.012
ma.S.L12      -0.1409      0.406     -0.347      0.729      -0.937       0.655
ma.S.L24      -0.8448      0.405     -2.084      0.037      -1.639      -0.050
sigma2         1.5340      0.025     61.288      0.000       1.485       1.583
===================================================================================
Ljung-Box (Q):                       40.99   Jarque-Bera (JB):             25657.62
Prob(Q):                              0.43   Prob(JB):                         0.00
Heteroskedasticity (H):               0.52   Skew:                             2.89
Prob(H) (two-sided):                  0.00   Kurtosis:                        20.05
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [0]:
train = t.iloc[0:1874]
test = t.iloc[1874:]

In [0]:
stepwise_model.fit(train)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(2, 1, 2, 12), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [0]:
future_forecast = stepwise_model.predict(n_periods=test.shape[0])
print(future_forecast)

[0.59289227 0.66375445 0.55693406 0.67156626 0.57208777 0.63773483
 0.6299083  0.6751338  0.7033775  0.57165908 0.65317031 0.67719847
 0.65487641 0.64703849 0.54607793 0.68008152 0.53588247 0.63763271
 0.66791382 0.68561799 0.72852107 0.64331885 0.62242469 0.64221009
 0.66154426 0.63954365 0.53774548 0.65011053 0.54926274 0.65690116
 0.65399688 0.69409876 0.72298625 0.68273133 0.62782461 0.65038825
 0.65787526 0.6515141  0.55023103]


In [0]:
future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['Prediction'])

In [0]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=future_forecast.index, y=future_forecast['Prediction'], showlegend=True,
                         mode='lines', name="Prediction",
                         marker=dict(color="blue")))
fig.add_trace(go.Scatter(x=test.index, y= test['FOODS_1_001_CA_1'], showlegend=True,
                         mode='lines', name="Test",
                         marker=dict(color="red")))
fig.update_layout(height=400, width=600)
fig.show()